In [1]:
class Dendrogram :
    def __init__(self,data,distance_type) :
        self.root = self.make_dendrogram(data,distance_type)
    
    #this function generates all leaf nodes(clusters) then find the two clusters with the shortes distance and merge them
    #and delete the two clusters from the list of clusters and repeat this process till just one cluster is left.
    def make_dendrogram(self,data,distance_type):
        clusters = []
        test_cluster = []
        for d in data :
            clusters.append(self.Cluster([d],0,None,None))
        while len(clusters) > 1:
            
            cluster_1 , cluster_2 , height = self.two_clusters_to_be_joined(clusters,distance_type)
            joint_cluster = self.Cluster(cluster_1.observations + cluster_2.observations,height,cluster_1,cluster_2)
            clusters.remove(cluster_1)
            clusters.remove(cluster_2)
            clusters.append(joint_cluster)
        return clusters[0]
    #this function finds the two clusters with the shorest distance and returns them along with the distance between them.
    def two_clusters_to_be_joined(self,clusters,distance_type) :
        from itertools import combinations
        
        #this line of code finds every combination (=pair) of 2 clusters.
        every_two_clusters = list(combinations(clusters,2))
        
        
        least_distance = float("inf")
        selected_clusters = []
        if distance_type=="complete_linkage" :
            for two_clusters in every_two_clusters :
                cluster_1,cluster_2 = list(two_clusters)
                distance=self.complete_distance(cluster_1,cluster_2)
                if distance < least_distance :
                    least_distance = distance 
                    selected_clusters = [cluster_1 , cluster_2]
                    
            return selected_clusters[0],selected_clusters[1],least_distance
        
        elif distance_type=="single_linkage" : 
            for two_clusters in every_two_clusters :
                cluster_1,cluster_2 = list(two_clusters)
                distance=self.single_distance(cluster_1,cluster_2)
                if distance < least_distance :
                    least_distance = distance 
                    selected_clusters = [cluster_1 , cluster_2]
                    
            return selected_clusters[0],selected_clusters[1],least_distance
        
        elif distance_type=="average_linkage" :
            for two_clusters in every_two_clusters :
                cluster_1,cluster_2 = list(two_clusters)
                distance=self.average_distance(cluster_1,cluster_2)
                if distance < least_distance :
                    least_distance = distance 
                    selected_clusters = [cluster_1 , cluster_2]
                    
            return selected_clusters[0],selected_clusters[1],least_distance
            
        elif distance_type=="centroid_linkage" :
            for two_clusters in every_two_clusters :
                cluster_1,cluster_2 = list(two_clusters)
                distance=self.centroid_distance(cluster_1,cluster_2)
                if distance < least_distance :
                    least_distance = distance 
                    selected_clusters = [cluster_1 , cluster_2]
                    
            return selected_clusters[0],selected_clusters[1],least_distance
            
        
    def complete_distance(self,A,B):
        from numpy import linalg,array
        from math import sqrt
        max_distance = 0
        for a in A.observations :
            for b in B.observations :
                distance = sqrt(sum((array(a)-array(b))**2))
                if distance > max_distance :
                    max_distance = distance
        return max_distance
    
    def single_distance(self,A,B):
        from numpy import linalg,array
        from math import sqrt
        min_distance = float("inf")
        for a in A.observations :
            for b in B.observations :
                distance = sqrt(sum((array(a)-array(b))**2))
                if distance < min_distance :
                    min_distance = distance
        return min_distance
    
    def average_distance(self,A,B):
        from numpy import array
        from math import sqrt
        sum_Of_distances = 0
        for a in A.observations :
            for b in B.observations :
                sum_Of_distances += sqrt(sum((array(a)-array(b))**2))
        
        return sum_Of_distances/(len(A.observations)*len(B.observations))
    
    def centroid_distance(self,A,B):
        from numpy import linalg,array,matrix
        from math import sqrt
        a = matrix(A.observations)
        b = matrix(B.observations)
        center_a  = a.mean(axis=0)
        center_b  = b.mean(axis=0)
        res = (center_a-center_b).tolist()[0]
        return sqrt(sum((array(res))**2))
                    
    class Cluster :
        def __init__(self,observations,height,L,R) :
            self.height = height
            self.observations = observations
            if len(observations)==1:
                self.isleaf = True
                self.height = height
            else :
                self.isleaf = False
                self.L = L
                self.R = R
        def __lt__(self, other):
           
            if self.height==0 :
                return False
            else:
                self_ = max(self.L.height,self.R.height)
                other_ = max(other.L.height,other.R.height)
                if self_>other_:
                    return True
                else :
                    return False
            
        


In [2]:
from numpy import genfromtxt,matrix
data = genfromtxt("ncidata.txt")
data = matrix.transpose(data)

In [3]:
dendrogram_comolete = Dendrogram(data,"complete_linkage")

In [4]:
dendrogram_single = Dendrogram(data,"single_linkage")

In [9]:
dendrogram_average = Dendrogram(data,"average_linkage")

In [10]:
dendrogram_centroid = Dendrogram(data,"centroid_linkage")

In [7]:
#a function used for getting required number of clusters from data
def getClusters(dendrogram,k) :
    from heapq import heappush,heappop
    queue = []
    heappush(queue,(-1*dendrogram.height,dendrogram))
    while len(queue) < k :
        priority,cluster = heappop(queue)
        heappush(queue,(-1*(cluster.L.height),cluster.L))
        heappush(queue,(-1*(cluster.R.height),cluster.R))
        
    return [item for priority,item in queue] 

In [14]:
import numpy as np
print("number of observations in each cluster using complete linkage for 3 clusters\n")

clusters =getClusters(dendrogram_comolete.root,3)

for cluster in clusters :
    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using single linkage for 3 clusters\n")

clusters =getClusters(dendrogram_single.root,3)
for cluster in clusters :

    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using average linkage for 3 clusters\n")

clusters =getClusters(dendrogram_average.root,3)
for cluster in clusters :
    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using centroid linkage for 3 clusters\n")
clusters =getClusters(dendrogram_centroid.root,3)
for cluster in clusters :

    print(len(cluster.observations))


number of observations in each cluster using complete linkage for 3 clusters

19
3
42
------------------------------------------------------------------------
number of observations in each cluster using single linkage for 3 clusters

62
1
1
------------------------------------------------------------------------
number of observations in each cluster using average linkage for 3 clusters

8
54
2
------------------------------------------------------------------------
number of observations in each cluster using centroid linkage for 3 clusters

61
1
2


In [15]:
print("number of observations in each cluster using complete linkage for 7 clusters\n")

clusters =getClusters(dendrogram_comolete.root,7)

for cluster in clusters :
    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using single linkage for 7 clusters\n")

clusters =getClusters(dendrogram_single.root,7)
for cluster in clusters :

    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using average linkage for 7 clusters\n")

clusters =getClusters(dendrogram_average.root,7)
for cluster in clusters :
    print(len(cluster.observations))

print("------------------------------------------------------------------------")
print("number of observations in each cluster using centroid linkage for 7 clusters\n")
clusters =getClusters(dendrogram_centroid.root,7)
for cluster in clusters :

    print(len(cluster.observations))


number of observations in each cluster using complete linkage for 7 clusters

8
3
11
2
9
10
21
------------------------------------------------------------------------
number of observations in each cluster using single linkage for 7 clusters

55
2
1
1
3
1
1
------------------------------------------------------------------------
number of observations in each cluster using average linkage for 7 clusters

13
2
41
1
1
1
5
------------------------------------------------------------------------
number of observations in each cluster using centroid linkage for 7 clusters

57
1
2
1
1
1
1
